### Timeline

Script para obtener archivo timeline-20-50-2-0.csv que contiene la estructura necesaria para representar el estado de los vehículos en el componente State timeline de Grafana.

In [1]:
import csv
import json
from datetime import datetime, timedelta

# Diccionario para mapear los valores originales a los nuevos valores
status_mapping = {
    'initVehicle': 'Iniciando vehículo',
    'movingEmptyVehicle': 'Vehículo libre en movimiento',
    'initJob': 'Iniciando trabajo',
    'movingWorkingVehicle': 'Vehículo trabajando en movimiento',
    'endingJob': 'Terminando trabajo',
    'finished': 'Trabajos completados'
}

# Leer el archivo JSON
with open('solutionsComplete/solutionComplete-20-50-2-0.json', 'r') as json_file:
    data = json.load(json_file)

# Obtener el número de vehículos
num_vehicles = len(data['routes'])

# Crear encabezados del CSV
fieldnames = ['timestamps'] + [f'Vehículo {i+1}' for i in range(num_vehicles)]

# Crear lista de filas
rows = []

# Fecha inicial
start_date = datetime(2000, 1, 1, 0, 0, 0)

# Iterar sobre cada trayectoria mixta
for i, trajectory in enumerate(data['mixedTrayectories']):
    # Calcular timestamp sumando minutos acumulados a la fecha inicial
    if trajectory['status'] == 'initVehicle':
        timestamp = datetime(2000, 1, 1, 0, 0, 0)  # Si el status es initVehicle, usar el timestamp 2000-01-01 00:00:00
    else:
        timestamp = start_date + timedelta(minutes=trajectory['accumulatedCost'])
    
    # Mapear el status a su nuevo valor
    status = status_mapping.get(trajectory['status'], trajectory['status'])
    
    # Crear fila
    row = {'timestamps': timestamp}  # Eliminamos el campo 'id'
    
    # Asignar status a la columna correspondiente al vehículo
    vehicle_index = trajectory['vehicle'] # Index basado en 0
    row[f'Vehículo {vehicle_index+1}'] = status
    
    # Agregar fila a la lista de filas
    rows.append(row)

    
# Eliminar filas donde aparece el valor 'waiting'
rows = [row for row in rows if 'waiting' not in row.values()]

# Iterar sobre las filas y ajustar los timestamps
current_vehicle = 0
for i in range(len(rows)):
    if f'Vehículo {current_vehicle+1}' in rows[i]:  # Verificar si la clave existe en la fila
        if rows[i][f'Vehículo {current_vehicle+1}'] != '':  # Si no es la primera fila de un vehículo
            if i < len(rows) - 1 and f'Vehículo {current_vehicle+1}' in rows[i+1] and rows[i+1][f'Vehículo {current_vehicle+1}'] != '':  # Si no es la última fila de un vehículo
                rows[i]['timestamps'] = rows[i]['timestamps']
            else:  # Si es la última fila de un vehículo
                rows[i]['timestamps'] = rows[i]['timestamps'] + timedelta(minutes=1)
        else:  # Si es la primera fila de un vehículo
            current_vehicle += 1
            rows[i]['timestamps'] = start_date

# Escribir datos en CSV
with open('timelines/timeline-20-50-2-0.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print("CSV creado satisfactoriamente.")


CSV creado satisfactoriamente.
